In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin

from tqdm.auto import tqdm
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
def load_data():
    """
    This function loads the data from the csv files and returns it as numpy arrays.

    input: None
    
    output: x_pretrain: np.ndarray, the features of the pretraining set
            y_pretrain: np.ndarray, the labels of the pretraining set
            x_train: np.ndarray, the features of the training set
            y_train: np.ndarray, the labels of the training set
            x_test: np.ndarray, the features of the test set
    """
    x_pretrain = pd.read_csv("./pretrain_features.csv.zip", index_col="Id", compression='zip').drop("smiles", axis=1).to_numpy()
    y_pretrain = pd.read_csv("./pretrain_labels.csv.zip", index_col="Id", compression='zip').to_numpy().squeeze(-1)
    x_train = pd.read_csv("./train_features.csv.zip", index_col="Id", compression='zip').drop("smiles", axis=1).to_numpy()
    y_train = pd.read_csv("./train_labels.csv.zip", index_col="Id", compression='zip').to_numpy().squeeze(-1)
    x_test = pd.read_csv("./test_features.csv.zip", index_col="Id", compression='zip').drop("smiles", axis=1)
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_pretrain = scaler.transform(x_pretrain)
    x_test_transed = scaler.transform(x_test)
    x_test[x_test.columns] = x_test_transed

    return x_pretrain, y_pretrain, x_train, y_train, x_test

In [4]:
class Net(nn.Module):
    """
    The model class, which defines our feature extractor used in pretraining.
    """
    def __init__(self):
        """
        The constructor of the model.
        """
        super().__init__()
        # TODO: Define the architecture of the model. It should be able to be trained on pretraing data 
        # and then used to extract features from the training and test data.
        self.fc1 = nn.Linear(1000, 1000)
        self.fc2 = nn.Linear(1000, 512)
        self.fc3 = nn.Linear(512, 512)
        self.fc4 = nn.Linear(512, 256)
        self.fc5 = nn.Linear(256, 256)
        self.fc6 = nn.Linear(256, 128)
        self.fc7 = nn.Linear(128, 64)
        self.fc8 = nn.Linear(64, 1)

        
        self.dropout1 = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.3)
        self.dropout3 = nn.Dropout(0.4)
        self.dropout4 = nn.Dropout(0.5)
        self.dropout5 = nn.Dropout(0.5)
        self.dropout6 = nn.Dropout(0.5)

        nn.init.xavier_normal_(self.fc1.weight)
        nn.init.xavier_normal_(self.fc2.weight)
        nn.init.xavier_normal_(self.fc3.weight)
        nn.init.xavier_normal_(self.fc4.weight)
        nn.init.xavier_normal_(self.fc5.weight)
        nn.init.xavier_normal_(self.fc6.weight)
        nn.init.xavier_normal_(self.fc7.weight)
        nn.init.xavier_normal_(self.fc8.weight)

    def forward(self, x):
        """
        The forward pass of the model.

        input: x: torch.Tensor, the input to the model

        output: x: torch.Tensor, the output of the model
        """
        # TODO: Implement the forward pass of the model, in accordance with the architecture 
        # defined in the constructor.
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        x = torch.relu(self.fc3(x))
        x = self.dropout3(x)
        x = torch.relu(self.fc4(x))
        x = self.dropout4(x)
        x = torch.relu(self.fc5(x))
        x = self.dropout5(x)
        x = torch.relu(self.fc6(x))
        x = self.dropout6(x)
        x = torch.relu(self.fc7(x))
        x = self.fc8(x)
        return x
    
    def make_feature(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        x = torch.relu(self.fc3(x))
        x = self.dropout3(x)
        x = torch.relu(self.fc4(x))
        x = self.dropout4(x)
        x = torch.relu(self.fc5(x))
        x = self.dropout5(x)
        x = torch.relu(self.fc6(x))
        return x

In [5]:
def make_feature_extractor(x, y, batch_size=256, eval_size=1000):
    """
    This function trains the feature extractor on the pretraining data and returns a function which
    can be used to extract features from the training and test data.

    input: x: np.ndarray, the features of the pretraining set
              y: np.ndarray, the labels of the pretraining set
                batch_size: int, the batch size used for training
                eval_size: int, the size of the validation set
            
    output: make_features: function, a function which can be used to extract features from the training and test data
    """
    # Pretraining data loading
    in_features = x.shape[-1]
    x_tr, x_val, y_tr, y_val = train_test_split(x, y, test_size=eval_size, random_state=0, shuffle=True)
    x_tr, x_val = torch.tensor(x_tr, dtype=torch.float), torch.tensor(x_val, dtype=torch.float)
    y_tr, y_val = torch.tensor(y_tr, dtype=torch.float), torch.tensor(y_val, dtype=torch.float)
    train_dataset = torch.utils.data.TensorDataset(x_tr, y_tr)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    val_dataset = torch.utils.data.TensorDataset(x_val, y_val)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # model declaration
    model = Net()
    model.to(device)
    model.train()
    
    # TODO: Implement the training loop. The model should be trained on the pretraining data. Use validation set 
    # to monitor the loss.
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=5, verbose=True)
    criterion = nn.MSELoss()
    epochs = 500
    for epoch in tqdm(range(epochs)):
        loss_tr = 0
        loss_val = 0
        for [x, y] in train_loader:
            optimizer.zero_grad()
            x = x.to(device)
            y = y.to(device)
            predictions = model(x).squeeze(-1)
            loss = criterion(predictions, y)
            loss.backward()
            optimizer.step()
            loss_tr += loss.item() * len(x)
        loss_tr /= len(train_loader.dataset)
        for [x, y] in val_loader:
            x = x.to(device)
            y = y.to(device)
            predictions = model(x).squeeze(-1)
            loss = criterion(predictions, y)
            loss_val += loss.item() * len(x)
        loss_val /= len(val_loader.dataset)
        scheduler.step(loss_val)
        print(f"Epoch {epoch+1}: train loss: {loss_tr}, val loss: {loss_val}")
        if(optimizer.param_groups[0]['lr'] < 1e-6):
            print(f"Early stop at epoch {epoch+1}")
            break

    def make_features(x):
        """
        This function extracts features from the training and test data, used in the actual pipeline 
        after the pretraining.

        input: x: np.ndarray, the features of the training or test set

        output: features: np.ndarray, the features extracted from the training or test set, propagated
        further in the pipeline
        """
        model.eval()
        # TODO: Implement the feature extraction, a part of a pretrained model used later in the pipeline.
        x = torch.tensor(x, dtype=torch.float)
        x = x.to(device)
        x = model.make_feature(x)
        return x

    return make_features

In [6]:
def make_pretraining_class(feature_extractors):
    """
    The wrapper function which makes pretraining API compatible with sklearn pipeline
    
    input: feature_extractors: dict, a dictionary of feature extractors

    output: PretrainedFeatures: class, a class which implements sklearn API
    """

    class PretrainedFeatures(BaseEstimator, TransformerMixin):
        """
        The wrapper class for Pretraining pipeline.
        """
        def __init__(self, *, feature_extractor=None, mode=None):
            self.feature_extractor = feature_extractor
            self.mode = mode

        def fit(self, X=None, y=None):
            return self

        def transform(self, X):
            assert self.feature_extractor is not None
            X_new = feature_extractors[self.feature_extractor](X)
            return X_new
        
    return PretrainedFeatures


In [7]:
def get_regression_model(X, y):

    class Model(nn.Module):
        """
        The model class, which defines our feature extractor used in pretraining.
        """
        def __init__(self):
            """
            The constructor of the model.
            """
            super().__init__()
            # TODO: Define the architecture of the model. It should be able to be trained on pretraing data 
            # and then used to extract features from the training and test data.
            self.fc1 = nn.Linear(128, 64)
            self.fc2 = nn.Linear(64, 64)
            self.fc3 = nn.Linear(64, 1)

            # nn.init.kaiming_uniform_(self.fc3.weight, nonlinearity='relu')
            nn.init.xavier_normal_(self.fc1.weight)
            nn.init.xavier_normal_(self.fc3.weight)
            nn.init.xavier_normal_(self.fc2.weight)

        def forward(self, x):
            """
            The forward pass of the model.

            input: x: torch.Tensor, the input to the model

            output: x: torch.Tensor, the output of the model
            """
            # TODO: Implement the forward pass of the model, in accordance with the architecture 
            # defined in the constructor.
            x = torch.relu(self.fc1(x))
            x = torch.relu(self.fc2(x))
            x = self.fc3(x)
            return x
    
    """
    This function returns the regression model used in the pipeline.

    input: None

    output: model: sklearn compatible model, the regression model
    """
    # TODO: Implement the regression model. It should be able to be trained on the features extracted
    # by the feature extractor.
    model = Model()
    model.to(device)
    model.train()

    # x = torch.tensor(X, dtype=torch.float)
    x = X.clone().detach()
    x = x.to(device)
    y = torch.tensor(y, dtype=torch.float)
    y = y.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=5, verbose=True)
    criterion = nn.MSELoss()
    epochs = 10000
    for epoch in tqdm(range(epochs)):
        optimizer.zero_grad()
        predictions = model(x).squeeze(-1)
        loss = criterion(predictions, y)
        loss.backward()
        optimizer.step()
        scheduler.step(loss)
        if(epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}: train loss: {loss}")
        if(optimizer.param_groups[0]['lr'] < 1e-7):
            print(f"Early stop at epoch {epoch+1}, loss: {loss}")
            break

    return model

In [8]:
result_file = "results-std-3-128-smallbatch.csv"

In [9]:
# Load data
x_pretrain, y_pretrain, x_train, y_train, x_test = load_data()
print("Data loaded!")
# Utilize pretraining data by creating feature extractor which extracts lumo energy 
# features from available initial features
feature_extractor =  make_feature_extractor(x_pretrain, y_pretrain, batch_size=32)
PretrainedFeatureClass = make_pretraining_class({"pretrain": feature_extractor})
pretrainedfeatures = PretrainedFeatureClass(feature_extractor="pretrain")

x_train_featured = pretrainedfeatures.transform(x_train)
x_test_featured = pretrainedfeatures.transform(x_test.to_numpy())
# regression model
regression_model = get_regression_model(x_train_featured, y_train)

y_pred = np.zeros(x_test.shape[0])
# TODO: Implement the pipeline. It should contain feature extraction and regression. You can optionally
# use other sklearn tools, such as StandardScaler, FunctionTransformer, etc.
y_pred = regression_model(x_test_featured).squeeze(-1).detach().cpu().numpy()

assert y_pred.shape == (x_test.shape[0],)
y_pred = pd.DataFrame({"y": y_pred}, index=x_test.index)
y_pred.to_csv(result_file, index_label="Id")
print(f"Predictions saved to {result_file}, all done!")

c:\Users\linan\.conda\envs\DL\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


Data loaded!


  0%|          | 0/500 [00:00<?, ?it/s]

Epoch 1: train loss: 0.2625987841316632, val loss: 0.05352412757277489
Epoch 2: train loss: 0.041118349831931446, val loss: 0.030246220111846925
Epoch 3: train loss: 0.022570555646200568, val loss: 0.01906789383292198
Epoch 4: train loss: 0.016902974290811286, val loss: 0.01704988967627287
Epoch 5: train loss: 0.014900950730455165, val loss: 0.01796092203259468
Epoch 6: train loss: 0.014766471628935969, val loss: 0.013808741204440594
Epoch 7: train loss: 0.014635348415344345, val loss: 0.015781521677970888
Epoch 8: train loss: 0.014455044816799309, val loss: 0.014000639252364635
Epoch 9: train loss: 0.014025708252678112, val loss: 0.02092244955897331
Epoch 10: train loss: 0.013507410859423025, val loss: 0.014035345688462257
Epoch 11: train loss: 0.013411084550831998, val loss: 0.015268064357340336
Epoch 00012: reducing learning rate of group 0 to 3.0000e-04.
Epoch 12: train loss: 0.013342080109429603, val loss: 0.01505603727698326
Epoch 13: train loss: 0.00896723657968093, val loss: 0.

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch 10: train loss: 2.518319606781006
Epoch 20: train loss: 1.884042739868164
Epoch 30: train loss: 1.1954892873764038
Epoch 40: train loss: 0.6302603483200073
Epoch 50: train loss: 0.3854168951511383
Epoch 60: train loss: 0.26691901683807373
Epoch 70: train loss: 0.16459307074546814
Epoch 80: train loss: 0.11049054563045502
Epoch 90: train loss: 0.08402150869369507
Epoch 100: train loss: 0.07226655632257462
Epoch 110: train loss: 0.06658436357975006
Epoch 120: train loss: 0.06332142651081085
Epoch 130: train loss: 0.06155552715063095
Epoch 140: train loss: 0.06063281372189522
Epoch 150: train loss: 0.05996851623058319
Epoch 160: train loss: 0.059500496834516525
Epoch 170: train loss: 0.059194087982177734
Epoch 180: train loss: 0.05900268256664276
Epoch 190: train loss: 0.05881177634000778
Epoch 200: train loss: 0.058639008551836014
Epoch 210: train loss: 0.05847782641649246
Epoch 220: train loss: 0.05831592530012131
Epoch 230: train loss: 0.058161020278930664
Epoch 240: train loss: 